# Unit 2 - Cost of Test

Group members:

- Ole Magnus Laerum
- Piotr Franc

# Task Description
Consider the dataset Breastcancer with the following partition train/test. Consider the test costs for each attribute given by the array testcosts (the first position of the array corresponds to the first attribute, CL.thickness, the second to Cell.size..). Consider also misclassification costs where the cost of predicting “benign” when the actual class is “malignant” is 20. The cost of predicting “malignant” when the actual class is “benign” is 4. Hits cost 0.

Try to minimise the global cost of your model. Draw a plot of test cost versus missclassification cost of the different options explored.

In [5]:
# install.packages("caret")
# install.packages("mlbench")
# install.packages("C50")
library(caret)
library(mlbench)

data(BreastCancer)
BreastCancer<-BreastCancer[,-1]
set.seed(480)
BreastCancer<-BreastCancer[complete.cases(BreastCancer),]
hv_index <- createDataPartition(BreastCancer$Class, p = .75, list = FALSE)
tr <- BreastCancer[ hv_index, ]
te <- BreastCancer[-hv_index, ]
testscosts<-c(2,4,3,1,5,3,2,4,2,0)
cost_FP = 4
cost_FN = 20

In [6]:
set.seed(144)
index <- createDataPartition(BreastCancer$Class, p = 0.8, list = FALSE)
train = BreastCancer[ index,]
test  = BreastCancer[-index,]

# Step 1

In [ ]:
create_model <- function(selected_indices){
selected_attributes <- names(BreastCancer)[selected_indices]
print(selected_attributes)
formula = as.formula(paste("Class ~", paste(selected_attributes, collapse = " + ")))
model <- train(formula, data = train, method = "C5.0")
return(model)
}

In [9]:
library(pROC)
library(caret)


for(i in 1:9){
    model = create_model(c(i))

    pred_probs <- predict(model, test, type = "prob")

    Class_numeric <- ifelse(test$Class == "malignant", 1, 0)

    preds <- predict(model, test)

    cm <- confusionMatrix(preds, test$Class)

    attributes_weights <- rep(0, length(testscosts))
    attributes_weights[selected_indices] <- 1
    cost = cm$table[3] * cost_FN + cm$table[2]*cost_FP + length(test) * sum(testscosts * attributes_weights)
    print(cost)
}


[1] "Cl.thickness"


ERROR: Error: object 'selected_indices' not found
